In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os
import tensorflow as tf
print(f"gpu: { len(tf.config.list_physical_devices('GPU')) }")
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)
from skimage.transform import resize
from tensorflow.keras import datasets


from tensorflow.keras.optimizers import Adam
import math
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from vae_injected import VAEInjected

gpu: 1


In [2]:
def load_data_for_model(model_name, batch_size=64):
    labels = np.load('./data/LFSD_labels.npy')
    depths = np.load('./data/LFSD_depths_repeated.npy')
    imgs = np.load('./data/LFSD_imgs.npy')
    masks = np.load('./data/LFSD_masks_single.npy')
    depths_feat = np.load('./data/LFSD_depths_repeated_%s_feat.npy' % model_name)
    imgs_feat = np.load('./data/LFSD_imgs_%s_feat.npy' % model_name)
    idx = np.random.permutation(len(labels))
    batch_idxs = [idx[i:i + batch_size] for i in range(0, len(labels), batch_size)]
    dataset = []
    for idx in batch_idxs:
        img_batch, depth_batch, mask_batch = imgs[idx], depths[idx], masks[idx]
        img_feat_batch, depth_feat_batch = imgs_feat[idx], depths_feat[idx]
        dataset.append((img_batch, img_feat_batch, depth_batch, depth_feat_batch, mask_batch))
    train_dataset, test_dataset = train_test_split(dataset, test_size=0.3)
    print("Train dataset contains %d batches of %d samples each" % (len(train_dataset), batch_size))
    print("Test dataset contains %d batches of %d samples each" % (len(test_dataset), batch_size))
    return train_dataset, test_dataset

In [3]:
def merge_images(image_batch, size):
    h,w = image_batch.shape[1], image_batch.shape[2]
    img = np.zeros((int(h*size[0]), w*size[1]))
    for idx, im in enumerate(image_batch):
        im = np.squeeze(im, axis=2)
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = im
    return img

In [4]:
def train_round(train_dataset, test_dataset, learning_rate, model_name, epochs):
    latent_lookup = {
        'inception': 2048,
        'efficientnet': 1280,
        'vgg': 512,
        'mobilenet': 1280,
        'resnet': 2048,
    }
    latent_dim = latent_lookup[model_name]
    vae = VAEInjected(latent_dim)
    vae.compile(optimizer=Adam(learning_rate))
    epochs = epochs
    # Training Step
    losses_across_epochs = {
        "loss": [],
        "reconstruction_loss": [],
        "kl_loss": [],
    }
    batch_num = len(train_dataset)
    for i in range(epochs):
       
        for k, v in losses_across_epochs.items():
            losses_across_epochs[k].append(0)
        for data in train_dataset:
            cur_loss = vae.train_step(data)
            for k, v in cur_loss.items():
                losses_across_epochs[k][-1] += cur_loss[k].numpy() / batch_num
            generated_image = vae.sample(data)
        print(f"Epoch {i} Total loss: { losses_across_epochs['loss'][-1]}")
        im_merged = merge_images(generated_image.numpy(), [8,8])
        plt.imsave('./images/vae_injected/%d.png' % i, im_merged, cmap='gray')
    for k, v in losses_across_epochs.items():
        np.save('./results/vae_injected/%s_%s' % (model_name, k), np.array(v))
    # Testing Step
    test_loss = 0
    for i, data in enumerate(test_dataset):
        _, _, _, _, mask_batch = data
        generated_image = vae.sample(data)
        reconstruction_loss = tf.reduce_sum(
            tf.keras.losses.binary_crossentropy(mask_batch, generated_image), [1,2]
        )
        test_loss += tf.reduce_mean(reconstruction_loss).numpy()
        im_merged = merge_images(generated_image.numpy(), [8,8])
        plt.imsave('./images/vae_injected/test_batch_%d.png' % i, im_merged, cmap='gray')
        
    test_loss = test_loss / len(test_dataset)
    np.save('./results/vae_injected/%s_test_loss' % model_name, np.array([test_loss]))
    return vae

In [5]:
def get_encoding_for_model(vae, model_name):

    train_dataset, test_dataset = None, None ## Freeing 
    (train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data()
    train_images = train_images / 255.0
    test_images = test_images / 255.0
    with tf.device('/cpu:0'):
        train_images = tf.image.resize(train_images, (256, 256)).numpy()
        test_images = tf.image.resize(test_images, (256, 256)).numpy()
    train_feats = np.load('./data/CIFAR100_%s_train_feat.npy' % model_name)
    test_feats = np.load('./data/CIFAR100_%s_test_feat.npy' % model_name)

    train_result, _, _ = vae.encode((train_images[:128], train_feats[:128], None, None, None), rand_depth=True)
    for i in tqdm(range(128, len(train_images), 128)):
        img = train_images[i:i+128]
        img_feat = train_feats[i:i+128]
        activation, _, _ = vae.encode((img, img_feat, None, None, None), rand_depth=True)
        train_result = tf.concat((train_result, activation), axis=0)
    np.save('./data/CIFAR100_vae_injected_%s_encoding_train.npy' % model_name, train_result.numpy())

    test_result, _, _ = vae.encode((test_images[:128], test_feats[:128], None, None, None), rand_depth=True)
    for i in tqdm(range(128, len(test_images), 128)):
        img = test_images[i:i+128]
        img_feat = test_feats[i:i+128]
        activation, _, _ = vae.encode((img, img_feat, None, None, None), rand_depth=True)
        test_result = tf.concat((test_result, activation), axis=0)
    np.save('./data/CIFAR100_vae_injected_%s_encoding_test.npy' % model_name, test_result.numpy())

In [6]:
%%time
learning_rate = 1e-4
epochs = 100

for model_name in ['efficientnet','inception', 'resnet', 'mobilenet', 'vgg']:
    train_dataset, test_dataset = load_data_for_model(model_name)
    trained_model = train_round(train_dataset, test_dataset, learning_rate, model_name, epochs)
    print("Gen encoding...")
    get_encoding_for_model(trained_model, model_name)

Train dataset contains 9 batches of 64 samples each
Test dataset contains 5 batches of 64 samples each
Epoch 0 Total loss: 3641427.888888889
Epoch 1 Total loss: 3220873.8333333335
Epoch 2 Total loss: 3116310.5
Epoch 3 Total loss: 3066843.25
Epoch 4 Total loss: 3027007.833333334
Epoch 5 Total loss: 2956550.8333333335
Epoch 6 Total loss: 2853031.4166666665
Epoch 7 Total loss: 2738564.805555556
Epoch 8 Total loss: 2624723.6944444445
Epoch 9 Total loss: 2517245.4166666665
Epoch 10 Total loss: 2418219.361111111
Epoch 11 Total loss: 2321482.027777778
Epoch 12 Total loss: 2234572.527777778
Epoch 13 Total loss: 2157736.388888889
Epoch 14 Total loss: 2082242.2222222222
Epoch 15 Total loss: 2009541.3333333337
Epoch 16 Total loss: 1941497.4166666667
Epoch 17 Total loss: 1878469.5555555557
Epoch 18 Total loss: 1818076.1805555555
Epoch 19 Total loss: 1760910.7222222222
Epoch 20 Total loss: 1705791.3333333333
Epoch 21 Total loss: 1653429.6805555555
Epoch 22 Total loss: 1604523.6944444445
Epoch 23 To

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 39.39it/s]


Train dataset contains 9 batches of 64 samples each
Test dataset contains 5 batches of 64 samples each
Epoch 0 Total loss: 3062518.6111111115
Epoch 1 Total loss: 2997835.9722222225
Epoch 2 Total loss: 2972191.7777777775
Epoch 3 Total loss: 2932727.5833333335
Epoch 4 Total loss: 2839278.25
Epoch 5 Total loss: 2707663.472222222
Epoch 6 Total loss: 2557740.027777778
Epoch 7 Total loss: 2391439.027777778
Epoch 8 Total loss: 2234871.8055555555
Epoch 9 Total loss: 2097507.625
Epoch 10 Total loss: 1977796.7638888892
Epoch 11 Total loss: 1868962.111111111
Epoch 12 Total loss: 1770880.0
Epoch 13 Total loss: 1682536.375
Epoch 14 Total loss: 1623829.1944444445
Epoch 15 Total loss: 1574405.6388888888
Epoch 16 Total loss: 1518828.8749999998
Epoch 17 Total loss: 1468627.875
Epoch 18 Total loss: 1421624.9444444445
Epoch 19 Total loss: 1376074.3888888888
Epoch 20 Total loss: 1332965.3888888888
Epoch 21 Total loss: 1292259.3472222222
Epoch 22 Total loss: 1254306.1527777778
Epoch 23 Total loss: 1218616.

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 61.46it/s]


Train dataset contains 9 batches of 64 samples each
Test dataset contains 5 batches of 64 samples each
Epoch 0 Total loss: 2959549.138888889
Epoch 1 Total loss: 2752441.333333333
Epoch 2 Total loss: 2676154.1944444445
Epoch 3 Total loss: 2573345.5833333335
Epoch 4 Total loss: 2451470.611111111
Epoch 5 Total loss: 2308682.027777778
Epoch 6 Total loss: 2157994.75
Epoch 7 Total loss: 2009383.5138888888
Epoch 8 Total loss: 1879501.763888889
Epoch 9 Total loss: 1765718.1666666665
Epoch 10 Total loss: 1663042.6805555555
Epoch 11 Total loss: 1573822.9583333335
Epoch 12 Total loss: 1532351.986111111
Epoch 13 Total loss: 1485052.5
Epoch 14 Total loss: 1424120.6944444445
Epoch 15 Total loss: 1370117.6944444445
Epoch 16 Total loss: 1320951.5
Epoch 17 Total loss: 1274491.986111111
Epoch 18 Total loss: 1233288.75
Epoch 19 Total loss: 1194405.166666667
Epoch 20 Total loss: 1157216.6666666667
Epoch 21 Total loss: 1121949.0694444445
Epoch 22 Total loss: 1088548.3472222222
Epoch 23 Total loss: 1057633.

...

Train dataset contains 9 batches of 64 samples each
Test dataset contains 5 batches of 64 samples each
Epoch 0 Total loss: 2843050.833333334
Epoch 1 Total loss: 2700839.055555555
Epoch 2 Total loss: 2664254.944444444
Epoch 3 Total loss: 2595385.583333333
Epoch 4 Total loss: 2488698.0
Epoch 5 Total loss: 2368003.0277777775
Epoch 6 Total loss: 2253285.0555555555
Epoch 7 Total loss: 2140539.847222222
Epoch 8 Total loss: 2031279.222222222
Epoch 9 Total loss: 1930344.972222222
Epoch 10 Total loss: 1834868.3333333333
Epoch 11 Total loss: 1747329.4166666665
Epoch 12 Total loss: 1667842.1388888888
Epoch 13 Total loss: 1596722.611111111
Epoch 14 Total loss: 1532683.375
Epoch 15 Total loss: 1474448.8472222222
Epoch 16 Total loss: 1420748.875
Epoch 17 Total loss: 1371207.6944444445
Epoch 18 Total loss: 1324631.513888889
Epoch 19 Total loss: 1280574.4305555557
Epoch 20 Total loss: 1239375.4583333333
Epoch 21 Total loss: 1201352.1805555555
Epoch 22 Total loss: 1165977.0
Epoch 23 Total loss: 1133723

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 72.34it/s]


Train dataset contains 9 batches of 64 samples each
Test dataset contains 5 batches of 64 samples each
Epoch 0 Total loss: 2689873.305555556
Epoch 1 Total loss: 2648610.722222222
Epoch 2 Total loss: 2642933.2777777775
Epoch 3 Total loss: 2616244.7222222225
Epoch 4 Total loss: 2557985.7222222225
Epoch 5 Total loss: 2474645.222222222
Epoch 6 Total loss: 2379699.2222222225
Epoch 7 Total loss: 2283353.722222222
Epoch 8 Total loss: 2187786.6944444445
Epoch 9 Total loss: 2094159.361111111
Epoch 10 Total loss: 2003392.7222222225
Epoch 11 Total loss: 1916182.2222222222
Epoch 12 Total loss: 1833134.2638888888
Epoch 13 Total loss: 1754675.4861111112
Epoch 14 Total loss: 1682511.5972222222
Epoch 15 Total loss: 1616266.1111111115
Epoch 16 Total loss: 1555544.1527777778
Epoch 17 Total loss: 1499824.6388888888
Epoch 18 Total loss: 1448896.2083333335
Epoch 19 Total loss: 1402102.9583333333
Epoch 20 Total loss: 1359177.2222222225
Epoch 21 Total loss: 1318833.2916666667
Epoch 22 Total loss: 1280390.444

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 72.80it/s]


Wall time: 15min 38s
